In [308]:
#imports 
import numpy as np
import pandas as pd
from scipy import stats # ANOVA
from numpy import linalg as LA
import matplotlib.pyplot as plt
import scipy as sp
%matplotlib inline

In [309]:
#constants
ALPHA = 0.85
EPSILON = 0.00001

In [310]:
# construct Adjacency matrix Z
Z = np.matrix('1 0 2 0 4 3; 3 0 1 1 0 0; 2 0 4 0 1 0; 0 0 1 0 0 1; 8 0 3 0 5 2; 0 0 0 0 0 0')
print('Adjcency Matrix Z\n', Z)
p = np.matrix('3 2 5 1 2 1')
print('Publish Number Matrix\n', p)
Z.shape

Adjcency Matrix Z
 [[1 0 2 0 4 3]
 [3 0 1 1 0 0]
 [2 0 4 0 1 0]
 [0 0 1 0 0 1]
 [8 0 3 0 5 2]
 [0 0 0 0 0 0]]
Publish Number Matrix
 [[3 2 5 1 2 1]]


(6, 6)

In [311]:
# set diagonals == 0.T
di = np.diag_indices(Z.shape[1])
Z[di] = 0
print('Set diagonals of Z == 0\n', Z)
print('\n', Z[::][:,2])
#list1 =np.array([[1,2,3,4,5,6]])
#print(list1.T)
#print(list1.shape)
#print(Z[:,2].shape)
#Z[:,2] = list1.T 
print(Z)

Set diagonals of Z == 0
 [[0 0 2 0 4 3]
 [3 0 1 1 0 0]
 [2 0 0 0 1 0]
 [0 0 1 0 0 1]
 [8 0 3 0 0 2]
 [0 0 0 0 0 0]]

 [[2]
 [1]
 [0]
 [1]
 [3]
 [0]]
[[0 0 2 0 4 3]
 [3 0 1 1 0 0]
 [2 0 0 0 1 0]
 [0 0 1 0 0 1]
 [8 0 3 0 0 2]
 [0 0 0 0 0 0]]


In [312]:
def norm(matrix):
    """This function normalizes a matrix. I wrote a function so it would run faster"""
    H = []
    count = 0
    for row in matrix:
        total_sum = matrix[:,count].sum()
        if total_sum == 0:
            total_sum = 1   
        H.append((1/total_sum) * matrix[:,count])
        count = count + 1
    H = np.array(H)
    return(H)
    

In [313]:
print(Z)
def norm1(matrix):    
    SUM = matrix.sum(axis = 0)
    matrix = matrix/SUM
    matrix[np.isnan(matrix)] = 0
    return(matrix)

[[0 0 2 0 4 3]
 [3 0 1 1 0 0]
 [2 0 0 0 1 0]
 [0 0 1 0 0 1]
 [8 0 3 0 0 2]
 [0 0 0 0 0 0]]


In [314]:
# Creating the normalized matrix H
H = norm1(Z)



print('Z normalized = H\n', H)



Z normalized = H
 [[0.         0.         0.28571429 0.         0.8        0.5       ]
 [0.23076923 0.         0.14285714 1.         0.         0.        ]
 [0.15384615 0.         0.         0.         0.2        0.        ]
 [0.         0.         0.14285714 0.         0.         0.16666667]
 [0.61538462 0.         0.42857143 0.         0.         0.33333333]
 [0.         0.         0.         0.         0.         0.        ]]


C:\Users\maryh\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.


In [315]:
# Identifying dangling nodes
def dangleNodes(matrix):
    """This function takes in a matrix and returns a vector of dangeling nodes"""
    index = []
    d= np.zeros(matrix.shape[0])
    index.append(np.where(~matrix.T.any(axis=1))[0])
    for i in index:
        np.put(d, [i], [1])
    return(d)
    

In [316]:
# dangleingNode array
d = dangleNodes(Z)
print('dangelingNode array d\n', d)

dangelingNode array d
 [0. 1. 0. 0. 0. 0.]


In [317]:
# constructing Article Vector
def Article(matrix):
    """computes article vector"""
    a = []
    matrix = matrix.T
    total_sum = matrix.sum()
    a.append((1/total_sum) * matrix[:,0])
    a = np.array(a)
    return(a)

In [318]:
a = Article(p)
a = np.matrix(a)
print(a.T)


[[0.21428571]
 [0.14285714]
 [0.35714286]
 [0.07142857]
 [0.14285714]
 [0.07142857]]


In [319]:
def InitStartVector(matrix):
    length = len(matrix)
    a = np.empty(length)
    a.fill(1/length)
    a = np.matrix(a)
    return(a)

In [320]:
# computing initial start vector
pi = InitStartVector(Z)
pi = pi.T
print(pi)

[[0.16666667]
 [0.16666667]
 [0.16666667]
 [0.16666667]
 [0.16666667]
 [0.16666667]]


In [321]:
# building matrix e
def getE(matrix):
    length = len(matrix)
    a = np.empty(length)
    a.fill(1)
    return(a)

In [322]:
e = getE(Z)
print(e)

[1. 1. 1. 1. 1. 1.]


In [323]:
# building a*e.T
aeT = a*e.T
print(aeT)

ValueError: shapes (1,6) and (1,6) not aligned: 6 (dim 1) != 1 (dim 0)

# replaceing columns of colummns corresponding with dangling nodes replaced with article vector
def replace_H(matrixH, matrixA, matrixD):
    index = 0
    a_list = matrixA.flatten()
    for row in matrixD:
        print(row)
        if np.not_equal(matrixD[index], [0]):
            print('inside if')
            matrixH[index,:] = a_list
        index += 1
        matrixH = matrixH.T
    return(matrixH)


H_T = H.T
H_prime = replace_H(H_T, a, d)
print(H_prime)

#np.where(~a.any(axis=1))[0]
print(H)

In [324]:
#Using the compute radpidly method to compute the influence vector

print(H.shape)
print(a)
print(pi)
def findP(H, pi, a):
    residual = 2
    count = 0
    while (EPSILON < residual):
        #pi1 = (ALPHA*d@pi + (1-ALPHA)) * a
        pi1 = (ALPHA*H*pi) + ((ALPHA*d@pi + (1-ALPHA)) * a).T
        residual = np.linalg.norm((pi1-pi), ord = 1)
        pi = pi1
        count = count+1
    print(count)
    return(pi)


(6, 6)
[[0.21428571 0.14285714 0.35714286 0.07142857 0.14285714 0.07142857]]
[[0.16666667]
 [0.16666667]
 [0.16666667]
 [0.16666667]
 [0.16666667]
 [0.16666667]]


In [325]:
#g = (ALPHA*d@pi + (1-ALPHA)) * a
a = findP(H, pi, a)
print(a)

18
[[0.30402138]
 [0.16360304]
 [0.18979616]
 [0.04661906]
 [0.27531309]
 [0.02064727]]


In [304]:
print(a)
print(H)

[[0.30402138]
 [0.16360304]
 [0.18979616]
 [0.04661906]
 [0.27531309]
 [0.02064727]]
[[0.         0.         0.28571429 0.         0.8        0.5       ]
 [0.23076923 0.         0.14285714 1.         0.         0.        ]
 [0.15384615 0.         0.         0.         0.2        0.        ]
 [0.         0.         0.14285714 0.         0.         0.16666667]
 [0.61538462 0.         0.42857143 0.         0.         0.33333333]
 [0.         0.         0.         0.         0.         0.        ]]


In [305]:
# finding Eigenfactor for each journal 
def Eigenfactor(matrixH, matrixa):
    dot = matrixH @ matrixa
    EF = np.array([dot / np.sum(dot)]) * 100
    #EF = np.array(EF[0])
    return(EF)
    

In [306]:
EF = Eigenfactor(H,a)
print(EF)
print(np.indices)

[[[34.05100649]
  [17.20374224]
  [12.17545523]
  [ 3.6531636 ]
  [32.91663244]
  [ 0.        ]]]
<function indices at 0x0DD80390>


In [307]:
import timeit

file = open('links.txt', 'r')

count = 0
Adj = np.empty((10748, 10748))
#Adj = sp.sparse.lil_matrix((10748, 10748))
for line in file:
    line = line.strip('\n')
    value = line.split(", ")
    r_list = list(map(int, value))
    Adj[value[0]],[value[1]] = value[2]
    count += 1

print(count)
        
f.close()



print(Adj)


ValueError: not enough values to unpack (expected 2, got 1)